<a href="https://colab.research.google.com/github/jaqueantunes/dados_saude/blob/main/app.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#app

In [ ]:
# --- BLOCO DE RESET E INICIALIZAÇÃO COMPLETA ---
import os

# 1. Instala as bibliotecas necessárias
print("⏳ Instalando Streamlit e Localtunnel...")
!pip install -q streamlit pandas plotly
!npm install -g localtunnel > /dev/null

# 2. Cria o arquivo app.py conectado ao SEU GitHub
print("📝 Criando o arquivo do aplicativo...")
codigo_app = """
import streamlit as st
import pandas as pd
import plotly.express as px

st.set_page_config(page_title="Dashboard Obesidade", layout="wide")
st.title("📊 Análise de Dados: Obesity")
st.markdown("Dados do repositório: [jaqueantunes/dados_saude](https://github.com/jaqueantunes/dados_saude)")

# Seu link do GitHub
url = "https://raw.githubusercontent.com/jaqueantunes/dados_saude/main/obesity.csv"

@st.cache_data
def carregar_dados():
    return pd.read_csv(url)

try:
    with st.spinner('Carregando dados...'):
        df = carregar_dados()
    st.success("Dados carregados com sucesso!")

    st.dataframe(df.head())

    st.divider()
    numeric_cols = df.select_dtypes(include=['number']).columns
    if len(numeric_cols) > 0:
        col = st.selectbox("Escolha uma coluna para o gráfico:", numeric_cols)
        st.plotly_chart(px.histogram(df, x=col), use_container_width=True)

except Exception as e:
    st.error(f"Erro: {e}")
"""

with open("app.py", "w") as f:
    f.write(codigo_app)

# 3. Mostra a SENHA (IP)
print("\n" + "="*30)
print("⚠️ COPIE ESTA SENHA (IP):")
!wget -q -O - ipv4.icanhazip.com
print("="*30 + "\n")

# 4. Inicia o Streamlit e o Túnel
print("🚀 Iniciando servidor... Clique no link que termina em .loca.lt abaixo!")
!streamlit run app.py & npx localtunnel --port 8501

⏳ Instalando Streamlit e Localtunnel...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 33.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 32.3 MB/s eta 0:00:00
npm notice
npm notice New major version of npm available! 10.8.2 -> 11.7.0
npm notice Changelog: https://github.com/npm/cli/releases/tag/v11.7.0
npm notice To update run: npm install -g npm@11.7.0
npm notice
📝 Criando o arquivo do aplicativo...

⚠️ COPIE ESTA SENHA (IP):
34.58.123.227

🚀 Iniciando servidor... Clique no link que termina em .loca.lt abaixo!
⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙

⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦
  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.58.123.227:8501

your url is: https://green-pugs-enter.loca.lt
